In [53]:
from xgboost import XGBClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier

import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import joblib
from datetime import datetime
import sys

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

input_file='D:\\burtm\\Visual_studio_code\\PD_related_projects\\outputs\\preprocessed_data\\icdar_train_df_KAGGLE_20250514_181737.csv'
sys.path.append(os.path.abspath("D:\\burtm\\Visual_studio_code\\PD_related_projects"))
# Load the CSV file into a DataFrame
train_FE = pd.read_csv(input_file)


In [54]:
# Count numeric columns
num_numeric_cols = train_FE.select_dtypes(include=['number']).shape[1]

# Count columns with missing values
num_missing_cols = train_FE.isnull().any().sum()

# Display results
print(f'Number of numeric columns: {num_numeric_cols}')
print(f'Number of columns with missing values: {num_missing_cols}')

Number of numeric columns: 7073
Number of columns with missing values: 0


In [55]:
print(train_FE.columns)

Index(['writer', 'page_id', 'same_text', 'tortuosityHist10[0]',
       'tortuosityHist10[1]', 'tortuosityHist10[2]', 'tortuosityHist10[3]',
       'tortuosityHist10[4]', 'tortuosityHist10[5]', 'tortuosityHist10[6]',
       ...
       'directions_hist1a2a3a4a5a6a7a8a9a10_220[214]',
       'directions_hist1a2a3a4a5a6a7a8a9a10_220[215]',
       'directions_hist1a2a3a4a5a6a7a8a9a10_220[216]',
       'directions_hist1a2a3a4a5a6a7a8a9a10_220[217]',
       'directions_hist1a2a3a4a5a6a7a8a9a10_220[218]',
       'directions_hist1a2a3a4a5a6a7a8a9a10_220[219]', 'isEng', 'train',
       'male', 'index'],
      dtype='object', length=7073)


In [56]:
train_FE.head()

,writer,page_id,same_text,tortuosityHist10[0],tortuosityHist10[1],tortuosityHist10[2],tortuosityHist10[3],tortuosityHist10[4],tortuosityHist10[5],tortuosityHist10[6],...,directions_hist1a2a3a4a5a6a7a8a9a10_220[214],directions_hist1a2a3a4a5a6a7a8a9a10_220[215],directions_hist1a2a3a4a5a6a7a8a9a10_220[216],directions_hist1a2a3a4a5a6a7a8a9a10_220[217],directions_hist1a2a3a4a5a6a7a8a9a10_220[218],directions_hist1a2a3a4a5a6a7a8a9a10_220[219],isEng,train,male,index
0,1,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003014,0.003225,0.003572,0.003957,0.004232,0.004804,0,1,0,0
1,1,2,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003246,0.003456,0.003709,0.003994,0.004308,0.004706,0,1,0,1
2,1,3,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003438,0.003732,0.004100,0.004329,0.004687,0.004879,1,1,0,2
3,1,4,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003788,0.004056,0.004285,0.004527,0.004843,0.005085,1,1,0,3
4,2,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003425,0.003403,0.003450,0.003626,0.003977,0.004547,0,1,0,4


In [57]:
#target_label='male'
target_label='isEng'
X_train = train_FE[train_FE['train']==1].drop(columns=['writer', 'same_text', 'train','page_id','isEng','train','index','male'])
y_train = train_FE[train_FE['train']==1][target_label]
X_val = train_FE[train_FE['train']==0].drop(columns=['writer', 'same_text', 'train','page_id','isEng','train','index','male'])
y_val = train_FE[train_FE['train']==0][target_label]

In [58]:
# Define the model
gbm_classifier = GradientBoostingClassifier(
    n_estimators=100,  
    learning_rate=0.1,  
    max_depth=3,  
    random_state=42
)

cat = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0, random_state=42)

lgbm = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=-1, random_state=42)

xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

feature_extraction_model = {'gbm': gbm_classifier, 'cat': cat, 'lgbm': lgbm, 'xgb': xgb}

In [59]:
selected_model = 'gbm'

In [60]:
import time

#feature_extraction_model = {'gbm': gbm_classifier, 'cat': cat, 'lgbm': lgbm, 'xgb': xgb}
# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Normalize features
    (selected_model, feature_extraction_model[selected_model])  # Train GBM classifier
])

In [61]:
# Measure the start time
start_time = time.time()

# Fit the model on training data
pipeline.fit(X_train, y_train)

# Measure the end time
end_time = time.time()

# Calculate the time taken
time_taken = end_time - start_time
print(f"Time taken to fit the model: {time_taken:.2f} seconds")

Time taken to fit the model: 218.06 seconds


In [62]:
# Predict on validation data
y_pred = pipeline.predict(X_val)

# Evaluate accuracy
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy:.4f}')

Validation Accuracy: 0.9741


In [63]:
# Predict on train data
y_pred = pipeline.predict(X_train)

# Evaluate accuracy
training_accuracy = accuracy_score(y_train, y_pred)
print(f'training Accuracy: {training_accuracy:.4f}')

training Accuracy: 1.0000


In [64]:
source_path = "D:\\burtm\\Visual_studio_code\\PD_related_projects"
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = os.path.join(source_path, "outputs", "models")
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

# Save the model
model_filename = os.path.join(output_dir, f"feature_extraction_model_{target_label}_{selected_model}_val{str(round(accuracy, 2)).replace('.', '-')}_{timestamp}.joblib")
joblib.dump(pipeline, model_filename)
print(f"Model saved to {model_filename}")

Model saved to D:\burtm\Visual_studio_code\PD_related_projects\outputs\models\feature_extraction_model_isEng_gbm_val0-97_20250514_193002.joblib


In [65]:
# Example usage:
LOG_FILE = output_dir+"\\file_metadata_log.json"
print(f"Log file path: {LOG_FILE}")
print(f"Output file path: {model_filename}")
file_IO.add_or_update_file(
    model_filename, LOG_FILE,
    custom_metadata={
        #"seed": seed,
        "trained on file": os.path.basename(input_file),
        "model": selected_model,
        "model_params": feature_extraction_model[selected_model].get_params(),
        "training time": time_taken,
        "accuracy train": training_accuracy,
        "accuracy val": accuracy,
        "problem type": target_label,
        "description": '''training information for the model''' 
    }
)

Log file path: D:\burtm\Visual_studio_code\PD_related_projects\outputs\models\file_metadata_log.json
Output file path: D:\burtm\Visual_studio_code\PD_related_projects\outputs\models\feature_extraction_model_isEng_gbm_val0-97_20250514_193002.joblib
Updated log for feature_extraction_model_isEng_gbm_val0-97_20250514_193002.joblib


In [66]:
file_IO.read_metadata(
    model_filename,
    log_path=LOG_FILE
)

Metadata for feature_extraction_model_isEng_gbm_val0-97_20250514_193002.joblib:
full_path: D:\burtm\Visual_studio_code\PD_related_projects\outputs\models\feature_extraction_model_isEng_gbm_val0-97_20250514_193002.joblib
size_bytes: 626421
created: 2025-05-14T19:30:02.242466
modified: 2025-05-14T19:30:02.372957
accessed: 2025-05-14T19:30:02.372957
trained on file: icdar_train_df_KAGGLE_20250514_181737.csv
model: gbm
model_params: {'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'log_loss', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'random_state': 42, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
training time: 218.05695796012878
accuracy train: 1.0
accuracy val: 0.9741379310344828
problem type: isEng
description: training

# easy access

In [30]:
def reload_modules():
    import importlib
    import utils.file_IO as file_IO
    

    importlib.reload(file_IO)

    return file_IO

file_IO = reload_modules()